## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002298DE39AF0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002298DE3A510>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Yuvraj and I am an Trainee Software Engineer @Valuebound")])

AIMessage(content="Hi Yuvraj! \n\nIt's great to meet you. That's awesome that you're a Trainee Software Engineer at Valuebound. \n\nWhat kind of projects are you working on? What are you most excited about learning during your training?  \n\nI'm always happy to chat about software engineering, if you'd like to bounce any ideas around or discuss something specific.\n", response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 33, 'total_tokens': 118, 'completion_time': 0.177683338, 'prompt_time': 0.004439771, 'queue_time': None, 'total_time': 0.182123109}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-06b8d39f-788f-4e43-836a-408bb686dc12-0', usage_metadata={'input_tokens': 33, 'output_tokens': 85, 'total_tokens': 118})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Yuvraj and I am an Trainee Software Engineer @Valuebound"),
        AIMessage(content="Hello Yuvraj! It's nice to meet you. \n\nAs a Trainee Software Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of Software Engineering.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You told me your name is Yuvraj and that you are a Trainee Software Engineer at Valuebound.  \n\nIs there something else you'd like to know or talk about? 😊  \n", response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 112, 'total_tokens': 155, 'completion_time': 0.087977816, 'prompt_time': 0.007874087, 'queue_time': None, 'total_time': 0.095851903}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-dfbd9695-0bb5-4948-9bf7-a5879c11454e-0', usage_metadata={'input_tokens': 112, 'output_tokens': 43, 'total_tokens': 155})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
!pip install langchain_community

  Using cached langchain_community-0.2.12-py3-none-any.whl.metadata (2.7 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached marshmallow-3.21.3-py3-none-any.whl.metadata (7.1 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain_community-0.2.12-py3-none-any.whl (2.3 MB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached marshmallow-3.21.3-py3-none-any.whl (49 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)


In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [8]:
config={"configurable":{"session_id":"chat1"}}

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Yuvraj and I am a Trainee Software Engineer")],
    config=config
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


In [12]:
response.content

"Hi Yuvraj! \n\nIt's great to meet another aspiring software engineer. What are you most excited about in your role as a Trainee?  \n\nIs there anything you're working on right now that you'd like to talk about? I'm always happy to hear about new projects and technologies. 😊  \n\n"

In [13]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content='Your name is Yuvraj. 😊  \n\nI remembered from our earlier conversation!  How can I help you today?  \n', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 210, 'total_tokens': 239, 'completion_time': 0.05839296, 'prompt_time': 0.012341062, 'queue_time': None, 'total_time': 0.07073402200000001}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-aa92daa0-3ba7-4d53-9e87-1d5870096419-0', usage_metadata={'input_tokens': 210, 'output_tokens': 29, 'total_tokens': 239})

In [14]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it! 😊\n"

In [16]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"Hi John! It's nice to meet you. 😊\n\nWhat can I do for you today?\n"

In [17]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Your name is John, remember?  😄  \n\nIs there anything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [19]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [20]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Yuvraj")]})

AIMessage(content="Hello Yuvraj! It's nice to meet you. \n\nHow can I help you today? 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 37, 'total_tokens': 65, 'completion_time': 0.056728297, 'prompt_time': 0.003008617, 'queue_time': None, 'total_time': 0.059736913999999995}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-4bfb2dd3-4a13-49d7-8293-d052b837dd25-0', usage_metadata={'input_tokens': 37, 'output_tokens': 28, 'total_tokens': 65})

In [21]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [22]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Yuvraj")],
    config=config
)

response

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content="Hi Yuvraj, it's nice to meet you! 👋  \n\nWhat can I do for you today? 😊  \n", response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 37, 'total_tokens': 66, 'completion_time': 0.058423449, 'prompt_time': 0.006581061, 'queue_time': None, 'total_time': 0.06500451}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d35e7ab2-4389-4849-bd47-21bd8df8415d-0', usage_metadata={'input_tokens': 37, 'output_tokens': 29, 'total_tokens': 66})

In [23]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Your name is Yuvraj,  you told me earlier! 😊  \n\nIs there anything else I can help you with?\n'

In [25]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [26]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Yuvraj")],"language":"Hindi"})
response.content

'नमस्ते य uvraj! 👋 \n\nमुझे आपसे बातचीत करने में खुशी हो रही है। 😊 \n\nआप क्या जानना चाहते हैं? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [27]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [28]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Yuvraj")],"language":"Hindi"},
    config=config
)
repsonse.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'नमस्ते, युवराज! 👋 \n\nआप कैसे हैं? 😊\n'

In [29]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


In [30]:
response.content

'आपका नाम युवराज है।  😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [37]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [38]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your ice cream preferences. \n\nWhat's your favorite flavor?  🍦😊\n"

In [39]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "What\'s 2 + 2?".  😊 \n'

In [40]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [41]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"As an AI, I don't have memory of past conversations or personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to use it! 😊  \n\n"

In [42]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is.  \n\n"